In [29]:
import pickle


with open("results/demos/1.pkl", "rb") as f:
    info = pickle.load(f)

message = info['prompt'][15][0].replace("""Please give your action in the following json format: \\{"action": an integer between 0 and 3, "justification": "justification of your choice."\\}""", "")

In [28]:
info['prompt'][15][1]

2

In [13]:
import os
from dataclasses import dataclass, field
from typing import Optional
import pickle
import glob
import torch
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments
import transformers
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_name = "./results/final_merged_checkpoint/"

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map={"": 0},
#     trust_remote_code=True,
#     use_auth_token=True,
# )
# base_model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda")




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
# message = """I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n"""

input_ids = tokenizer(message, return_tensors="pt")["input_ids"].to("cuda")
output = model.generate(
    input_ids,
    max_new_tokens=200,
)
output = output[0].to("cpu")


filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
print(filling)    



Answer: 3


In [6]:
import glob
import pickle
from datasets import Dataset
prompt_list = []
action_list = []
for file_name in glob.glob("results/demos/*.pkl"):
    with open(file_name, "rb") as f:
        dd = pickle.load(f)
    for prompt, action, justification in dd['prompt']:
        prompt_list.append(prompt)
        action_list.append(action)

In [9]:
import numpy as np
np.max([len(p) for p in prompt_list])

9312

In [7]:
from huggingface_hub import hf_hub_download
# /home/lesong/.cache/huggingface/modules/transformers_modules/baichuan-inc/Baichuan2-7B-Chat-4bits/07a1fadfa7741d89789474e2244123f4d793842b/
hf_hub_download(repo_id="baichuan-inc/Baichuan2-13B-Chat-4bits", filename="pytorch_model.bin", force_download=True)


'/home/lesong/.cache/huggingface/hub/models--baichuan-inc--Baichuan2-13B-Chat-4bits/snapshots/9b37dd813b19a11925cfe57ad322e602f83298af/pytorch_model.bin'

In [2]:
!pip install huggingface_hub

     |████████████████████████████████| 294 kB 5.7 MB/s eta 0:00:01
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
     |████████████████████████████████| 173 kB 100.4 MB/s eta 0:00:01


In [ ]:
import os
from dataclasses import dataclass, field
from typing import Dict, Optional
import pickle

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments, AutoModelForCausalLM
from transformers import pipeline
import pickle





# print(prompt)


import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "meta-llama/Llama-2-13b-hf"
adapters_name = "./results/checkpoint-500"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 3
stop_token_ids = [0]

: 

In [ ]:
data_dir = "demo_pairs/demo_99.pkl"
with open(data_dir, "rb") as f:
    data_dict = pickle.load(f)
    total_samples = len(data_dict["prompt"])
    
prompt = data_dict["prompt"][200]

inputs = tokenizer(
    prompt,
    return_tensors="pt",
)

device = torch.device("cuda:0")
m = m.to(device)

print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = m.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=3, eos_token_id=3, bos_token_id=1,
        return_dict_in_generate=True, output_scores=True)
    
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    # generated_sequences = outputs.sequences[:, inputs["input_ids"].shape[-1]:]
    # probabilities = torch.stack(outputs.scores, dim=1).softmax(-1)
    # texts_probabilities = torch.gather(probabilities, 2, generated_sequences[:, :, None]).squeeze(-1)
    # _scores = texts_probabilities.prod(-1)


: 

In [ ]:
_scores

: 